In [ ]:
%load_ext autoreload
%autoreload 2

<div align="center">
    <hr> 
  <span style="font-family: Arial, sans-serif; font-size: 24px; font-weight: bold;">
    M2 - IPParis - Advanced Experimental Methods in Fluid Mechanics Acquisition - Signal processing
  </span>
  <hr> 
  <span style="font-family: Arial, sans-serif; font-size: 18px;"> 
    <span style="font-weight: bold;">Instructor:</span> Romain Monchaux - romain.monchaux@ensta.fr
  </span>
    <br>
    <br>
    <span style="font-family: Arial, sans-serif; font-size: 18px;"> 
        <span style="font-weight: bold;">Affiliation:</span> ENSTA-Paris - Institut Polytechnique de Paris
    </span>
    <hr> 
</div>

<div align="left">
    <hr> 
  <span style="font-family: Arial, sans-serif; font-size: 24px; font-weight: bold;">
    3 - Real signal analysis
  </span>
</div>

## Context

We are going to analyze here various real signals by using the statistical quantities that we have just discovered on the synthetic signals.

<div align="center">
    <hr> 
</div>

## Common libraries and functions

In [ ]:
## Import necessary functions for all operations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scipy.io
from scipy.signal import welch as welch
from scipy.io import wavfile as wavfile

In [ ]:
##########################################
## Let us create the re-usable functions here
##########################################

## Function 1: Gaussian Probability Density Function (PDF)
def Gauss(x):
    """
    Calculates the value of the standard normal (Gaussian) PDF at a point x.
    Formula: f(x) = (1 / sqrt(2*pi)) * exp(-x^2 / 2)
    This function is useful for plotting a theoretical Gaussian distribution.
    """
    return (1 / np.sqrt(2 * np.pi)) * np.exp(-(x ** 2) / 2)

## Function 2: Estimating a PDF from a Signal
def Calc_PDF(SIG, NbBin):
    """
    Estimates the Probability Density Function (PDF) of a signal.
    
    This is done by creating a histogram and normalizing it so that the
    total area under the histogram is equal to 1. The `density=True` option
    in `np.histogram` handles this normalization automatically, which is
    more efficient and less prone to error than manual normalization.
    
    Args:
        SIG (numpy.ndarray): The input signal (a 1D array of numerical data).
        NbBin (int): The number of bins to use for the histogram.
        
    Returns:
        tuple: A tuple containing two numpy arrays:
               - Bin_Centers (numpy.ndarray): The center values of each bin.
               - PDF (numpy.ndarray): The estimated PDF values for each bin.
    """
    # Use np.histogram with the 'density=True' option.
    # This directly returns the PDF values for each bin.
    PDF, Bin_Edges = np.histogram(SIG, bins=NbBin, density=True)
    
    # Calculate the center of each bin for plotting.
    # np.diff() calculates the difference between consecutive bin edges.
    # The result is a simple and accurate way to find the center of each bar.
    Bin_Centers = Bin_Edges[:-1] + np.diff(Bin_Edges) / 2
    
    return Bin_Centers, PDF

## 3.1 - Turbulent jet

We are first interested in the signal of a turbulent jet captured by a hot wire. This one measures the longitudinal velocity component Ux at the jet axis at about 60 diameters after the nozzle exit.

<div align="center">
    <hr> 
</div>

### Question - 1

Follow the instructions below,

1. Load the signaljet file.
2. Plot the velocity signal as a function of time. Zoom in on different time horizons and centered on different instants.
3. Comment on the stationarity of the signal and its frequency content from these visualizations.
4. Compute the mean ($\overline{U_x}$) and standard deviation ($u_{x_{rms}}$) of the velocity over time intervals of different lengths.
5. Comment on the differences obtained. Is the signal stationary?

In [ ]:
# load signal

data = pd.read_csv("../data/signaljet", sep = "\t", header = None)
data.columns = ["time", "signal"]

time = data["time"].to_list()
signal = data["signal"].to_list()

# Sampling frequency
Fs = 10000
Nsamples = len(time)
duration = Nsamples / Fs

In [ ]:
##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(time, signal, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"U_x (m/s)", fontsize = 14, labelpad = 5)
ax.set_title(f'Signal of duration {duration}s sampled at Fs={Fs/1000}kHz', fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()
plt.savefig(f'./figures/jet.png',  bbox_inches = 'tight')

In [ ]:
# Zooming to identify as much things as possible
# zoomed figure
## Define your limits here
##########################################
x0 = 10
dx = 0.01
##########################################

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(time, signal, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel("Signal", fontsize = 14, labelpad = 5)
ax.set_title(f'Signal of duration {duration}s sampled at Fs={Fs/1000}kHz', fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xlim([x0,x0+dx])
plt.show()
##########################################
plt.savefig(f'./figures/jet_zoom_'+str(xmax)+'.png',  bbox_inches = 'tight')

In [ ]:
##########################################################################
# First statistical moments estimated over 1s 
###########################################################################

# sliding interval length (s)
Tint = 0.01
# number of intervals
Nintervals = round(duration/Tint)
# time vector
t1s = np.linspace(0, Nintervals * Tint, Nintervals)

# initialise averages and standard deviations
mean_U = np.zeros(Nintervals)
std_U = np.zeros(Nintervals)
# number of samples per interval
Nsamples_int = int(Tint*Fs)

# loop on intervals
for i in range(1,Nintervals): 
    mean_U[i] = np.mean(signal[(i-1) * Nsamples_int + 1:i * Nsamples_int])
    std_U[i] = np.std(signal[(i-1) * Nsamples_int + 1:i * Nsamples_int])

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(t1s, mean_U, marker = "o", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="red", color="red", label = "average")

ax.plot(t1s, std_U, marker = "s", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="blue", color="blue", label = "standard deviation")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel("Velocity statistics (m/s)", fontsize = 14, labelpad = 5)
ax.set_title(f"Sliding moments over {Tint}s", fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()
plt.savefig(f'./figures/jet_moy{1000*Tint}.png',  bbox_inches = 'tight')

### Question - 2

Calculate the velocity signal spectrum for the full signal.

1. What is the frequency resolution of this spectrum?
2. Comment on the obtained spectrum, in particular its bounds

In [ ]:
# Full signal, no average
Nfft = Nsamples
# frequency resolution
df = Fs/Nfft
Nwindow = Nfft
Noverlap = Nfft/2
# Spectrum estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={Fs/Nfft:.3f}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(loc = "upper right", fontsize = 18)

ax.grid()

plt.show()

##########################################
## Saving figure here
##########################################
fig.savefig(f'./figures/jet_freq_{Nfft}.png',  bbox_inches = 'tight')

### Question - 3

Calculate the spectrum of the velocity signal for different window lengths (i.e. frequency resolutions).

1. Comment the obtained spectra and compare them to the one obtained for the full signal.

<div style="background-color: #e0f2f1;">
  <span style="font-weight: bold;">Hint:</span> Here we have only three frequency cells. But, if you want more, copy and paste the codes in the cell to another cell to produce more!
</div>

#### Frequency - 1

In [ ]:
# Frequency resolution XX Hz
df = XX
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={Fs/Nfft:.3f}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

#### Frequency - 2

In [ ]:
# Frequency resolution XX Hz
df = XX
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={Fs/Nfft:.3f}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

#### Frequency - 3

In [ ]:
# Frequency resolution XX Hz
df = XX
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={Fs/Nfft:.3f}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

### Question - 4

Plot the auto-correlation and PDF of the jet signal. Discuss the results obtained. Compare to the synthetic signals from the previous exercise.

#### PDF observation

In [ ]:
Sig = (signal-np.mean(signal))/np.std(signal)
NbPt = len(signal)
NbBin = 30

Bin,PDF = Calc_PDF(Sig[:NbPt], NbBin)
BG = np.linspace(-5,5,2**20)

##########################################
## Plotting routine
##########################################
# Create a figure
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(15, 5))

ax[0].plot(Bin, PDF, linewidth=2, color="tab:blue", linestyle="-", label = "Bin vs PDF") 
ax[0].plot(BG, Gauss(BG), linewidth=2, color="black", linestyle='-.', label = "BG vs Gauss(BG)")
ax[0].set_xlabel('U (m/s)', fontsize=14, labelpad=5)
ax[0].set_ylabel('PDF', fontsize=14, labelpad=5)
ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].tick_params(axis='both', which='minor', labelsize=12)
ax[0].set_title(f'PDF of turbulent jet Linear Scale')
ax[0].grid()
ax[0].legend(loc = "upper right")

ax[1].plot(Bin, PDF, linewidth=2, color="tab:blue", linestyle="-", label = "Bin vs PDF") 
ax[1].plot(BG, Gauss(BG), linewidth=2, color="black", linestyle='-.', label = "BG vs Gauss(BG)")
ax[1].set_xlabel('U (m/s)', fontsize=14, labelpad=5)
ax[1].set_ylabel('PDF', fontsize=14, labelpad=5)
ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='minor', labelsize=12)
ax[1].set_title(f'PDF of turbulent jet Log Scale')
ax[1].set_yscale('log')
ax[1].grid()
ax[1].legend(loc = "upper right")

# Adjust layout and save the figure
plt.tight_layout()

fig.savefig(f'./figures/jet_PDF.png',  bbox_inches = 'tight')

#### Autocorrelation

In [ ]:
##########################################
## Main calculations are here
##########################################
test0 = signal[:100000]
test = test0-np.mean(test0)
testnorm = np.sum(test**2)
acor = np.correlate(test,test,mode="same")/testnorm
acor = acor[len(acor)//2:]

##########################################
## Plotting routine
##########################################

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

delta_t=np.linspace(0,len(acor)/Fs,len(acor))

ax.plot(delta_t, acor, marker = "o", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel(rf'$\Delta$ t', fontsize = 14, labelpad = 5)
ax.set_ylabel('Normalised autocorrelation', fontsize = 14, labelpad = 5)
ax.set_title(f"Turbulent jet: Autocorrelation", fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()

ax.set_xlim(-0.0001,0.2)
    
plt.show()

##########################################
## Saving figure here
##########################################
fig.savefig(f'./figures/jet_autocorrelation.png',  bbox_inches = 'tight')

## 3.2 - Wake

We are now interested in the signal of a turbulent wake captured by a hot wire. The hot wire measures the longitudinal velocity component $U_x$ at the wake axis at about 10 diameters after the cylinder.


<div align="center">
    <hr> 
</div>

### Question - 1

Follow the instructions below,

1. Load the signal_cylindre.txt file.
2. Plot the velocity signal as a function of time. Zoom in on different time horizons and centered on different instants.
3. Comment on the stationarity of the signal and its frequency content from these visualizations.
4. Compute the mean ($\overline{U_x}$) and standard deviation ($u_{x_{rms}}$) of the velocity over time intervals of different lengths.
5. Comment on the differences obtained. Is the signal stationary?

In [ ]:
# load signal

data = pd.read_csv("../data/signal_cylindre.txt", sep = "\t", header = None)
data.columns = ["time", "signal", "aux1", "aux2"]

time = data["time"].to_list()
signal = data["signal"].to_list()

# Sampling frequency
Fs = 1000
time = np.linspace(0,len(signal)/Fs,len(signal))
Nsamples = len(time)
duration = Nsamples/Fs

In [ ]:
##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(time, signal, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"U_x (m/s)", fontsize = 14, labelpad = 5)
ax.set_title(f'Signal of duration {duration}s sampled at Fs={Fs/1000}kHz', fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()
plt.savefig(f'./figures/wake.png',  bbox_inches = 'tight')

In [ ]:
# Zooming to identify as much things as possible
# zoomed figure
## Define your limits here
##########################################
x0 = 
dx = 
##########################################

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(time, signal, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel("Signal", fontsize = 14, labelpad = 5)
ax.set_title(f'Signal of duration {duration}s sampled at Fs={Fs/1000}kHz', fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xlim([x0,x0+dx])
plt.show()
##########################################
plt.savefig(f'./figures/wake_zoom_'+str(1000*dx)+'.png',  bbox_inches = 'tight')

In [ ]:
##########################################################################
# First statistical moments estimated over 1s 
###########################################################################

# sliding interval length (s)
Tint = 0.01
# number of intervals
Nintervals = round(duration/Tint)
# time vector
t1s = np.linspace(0, Nintervals * Tint, Nintervals)

# initialise averages and standard deviations
mean_U = np.zeros(Nintervals)
std_U = np.zeros(Nintervals)
# number of samples per interval
Nsamples_int = int(Tint*Fs)

# loop on intervals
for i in range(1,Nintervals): 
    mean_U[i] = np.mean(signal[(i-1) * Nsamples_int + 1:i * Nsamples_int])
    std_U[i] = np.std(signal[(i-1) * Nsamples_int + 1:i * Nsamples_int])

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

ax.plot(t1s, mean_U, marker = "o", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="red", color="red", label = "average")

ax.plot(t1s, std_U, marker = "s", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="blue", color="blue", label = "standard deviation")

ax.set_xlabel("Time (s)", fontsize = 14, labelpad = 5)
ax.set_ylabel("Velocity statistics (m/s)", fontsize = 14, labelpad = 5)
ax.set_title(f"Sliding moments over {Tint}s", fontsize = 14, pad = 20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()
plt.savefig(f'./figures/jet_moy{1000*Tint}.png',  bbox_inches = 'tight')

### Question - 2

You are hereby tasked to answer,

1. Calculate the spectrum of the velocity signal using different window lengths (i.e. frequency resolutions).
2. Calculate the PDF and the auto-correlation of the signal.
3. Comment on the different results obtained

#### Full signal, no average

In [ ]:
Nfft = Nsamples
# frequency resolution
df = Fs/Nfft
Nwindow = Nfft
Noverlap = Nfft/2

## Let us calculate using welch
F, Pxx = welch(np.array(signal[:Nwindow]), 
               Fs, 
               window = 'hamming', 
               nperseg = Nwindow, 
               noverlap = Noverlap, 
               nfft = Nfft)


##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df:.3f}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf'DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)', fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent wake', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.legend(loc = "upper right", fontsize = 18)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

plt.show()

##########################################
## Saving figure here
##########################################
fig.savefig(f'./figures/wake_freq_{Nfft}.png',  bbox_inches = 'tight')

#### Frequency - 1

In [ ]:
# Frequency resolution XX Hz
df = 0.05
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

#### Frequency - 2

In [ ]:
# Frequency resolution XX Hz
df = 0.5
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

#### Frequency - 3

In [ ]:
# Frequency resolution XX Hz
df = 5
Nfft = int(Fs/df)
Noverlap = Nfft/2
Nwindow = Nfft

# Estimation
F,Pxx = welch(np.array(signal[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,7))

ax.plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df}")

ax.set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax.set_ylabel(rf"DSP (m$^2\cdot$s$^{-2}\cdot$Hz$^{-1}$)", fontsize = 14, labelpad = 5)
ax.set_title('Power Spectral Density: turbulent jet', fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.set_xscale("log")
ax.set_yscale("log")

ax.grid()

ax.legend(loc = "upper right", fontsize = 18)

fig.savefig(f'./figures/jet_freq_{df}_{Nfft}.png',  bbox_inches = 'tight')

#### PDF observation

In [ ]:
Sig = (signal-np.mean(signal))/np.std(signal)
NbPt = len(signal)
NbBin = 30

Bin,PDF = Calc_PDF(Sig[:NbPt], NbBin)
BG = np.linspace(-5,5,2**20)

##########################################
## Plotting routine
##########################################
# Create a figure
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(15, 5))

ax[0].plot(Bin, PDF, linewidth=2, color="tab:blue", linestyle="-", label = "Bin vs PDF") 
ax[0].plot(BG, Gauss(BG), linewidth=2, color="black", linestyle='-.', label = "BG vs Gauss(BG)")
ax[0].set_xlabel('U (m/s)', fontsize=14, labelpad=5)
ax[0].set_ylabel('PDF', fontsize=14, labelpad=5)
ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].tick_params(axis='both', which='minor', labelsize=12)
ax[0].set_title(f'PDF of turbulent jet Linear Scale')
ax[0].grid()
ax[0].legend(loc = "upper right")

ax[1].plot(Bin, PDF, linewidth=2, color="tab:blue", linestyle="-", label = "Bin vs PDF") 
ax[1].plot(BG, Gauss(BG), linewidth=2, color="black", linestyle='-.', label = "BG vs Gauss(BG)")
ax[1].set_xlabel('U (m/s)', fontsize=14, labelpad=5)
ax[1].set_ylabel('PDF', fontsize=14, labelpad=5)
ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='minor', labelsize=12)
ax[1].set_title(f'PDF of turbulent jet Log Scale')
ax[1].set_yscale('log')
ax[1].grid()
ax[1].legend(loc = "upper right")

# Adjust layout and save the figure
plt.tight_layout()

fig.savefig(f'./figures/wake_PDF.png',  bbox_inches = 'tight')

#### Autocorrelation

In [ ]:
##########################################
## Main calculations are here
##########################################
test0 = signal[:100000]
test = test0-np.mean(test0)
testnorm = np.sum(test**2)
acor = np.correlate(test,test,mode="same")/testnorm
acor = acor[len(acor)//2:]

##########################################
## Plotting routine
##########################################

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

delta_t=np.linspace(0,len(acor)/Fs,len(acor))

ax.plot(delta_t, acor, marker = "o", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue")

ax.set_xlabel(rf'$\Delta$ t', fontsize = 14, labelpad = 5)
ax.set_ylabel('Normalised autocorrelation', fontsize = 14, labelpad = 5)
ax.set_title(f"Turbulent jet: Autocorrelation", fontsize=14, pad=20)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=12)

ax.grid()

ax.set_xlim(-0.0001,0.2)
    
plt.show()

##########################################
## Saving figure here
##########################################
fig.savefig(f'./figures/wake_autocorrelation.png',  bbox_inches = 'tight')

## 3.3 - Gong signal

We are now interested in the pressure signal produced by a gong forced by a coil-magnet system oscillating sinusoidally at a given frequency $f_0$.

<div align="center">
    <hr> 
</div>

### Question - 1

Load the file gong_force.wav using scipy.io.wavfile.read. This function also returns the sampling frequency and allows to get the number of bits used for discretisation.

1. Plot the pressure signal as well as the forcing signal against time.
2. Calculate the mean (overlinep) and standard deviation ($p_rms$) of the pressure over 1 second intervals.
3. Is the signal stationary?

#### Loading signal and observation

In [ ]:
# load signal
Fs,signal = scipy.io.wavfile.read('../data/gong_force.wav')
nbits = signal.dtype

Ts = 1/Fs # time step
Nsamples = len(signal) # sample number
time = np.linspace(0,Nsamples*Ts,Nsamples) # time vector
duration = round(Nsamples*Ts,1) # signal duration

# Split forcing/response
Fsig = signal[:,0]
Psig = signal[:,1]


##########################################
## Plotting routine
##########################################

fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

ax[0].plot(time, Fsig, marker = "o", markersize=1, linewidth = 1, linestyle = "-",
        markerfacecolor = "tab:blue", markeredgecolor="black", color="black")

ax[0].set_xlabel(rf'Time (s)', fontsize = 14, labelpad = 5)
ax[0].set_ylabel(f'Forcing Amplitude (AU)', fontsize = 14, labelpad = 5)
ax[0].set_title(f'Signal of duration {duration}s sampled at Fs={Fs/1000}kHz', fontsize=14, pad=20)

ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].tick_params(axis='both', which='minor', labelsize=12)

ax[0].grid()

ax[1].plot(time, Psig, marker = "o", markersize=1, linewidth = 1, linestyle = "-",
        markerfacecolor = "tab:blue", markeredgecolor="black", color="black")

ax[1].set_xlabel(rf'Time (s)', fontsize = 14, labelpad = 5)
ax[1].set_ylabel('Pressure (Pa)', fontsize = 14, labelpad = 5)

ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='minor', labelsize=12)

ax[1].grid()


##########################################
## Saving figure here
##########################################
fig.savefig(f'./figures/gong.png',  bbox_inches = 'tight')

#### First statistical moment

In [ ]:
# sliding interval length (s)
Tint = 1
# Number of intervals
Nintervals = round(duration/Tint)
# time vector 
t1s = np.linspace(0,Nintervals*Tint,Nintervals)

# initialise averages and standard deviations
mean_f1s = np.zeros(Nintervals)
std_f1s = np.zeros(Nintervals)
mean_p1s = np.zeros(Nintervals)
std_p1s = np.zeros(Nintervals)
# number of samples per interval
Nsamples_int = int(Tint*Fs) 

# loop on intervals
for i in range(1,Nintervals): 
    mean_f1s[i] = np.mean(Fsig[(i-1)*Nsamples_int+1:i*Nsamples_int])
    std_f1s[i] = np.std(Fsig[(i-1)*Nsamples_int+1:i*Nsamples_int])
    mean_p1s[i] = np.mean(Psig[(i-1)*Nsamples_int+1:i*Nsamples_int])
    std_p1s[i] = np.std(Psig[(i-1)*Nsamples_int+1:i*Nsamples_int])

##########################################
## Plotting routine
##########################################

fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

ax[0].plot(t1s, mean_f1s, marker = "o", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "black", markeredgecolor="black", color="black", label = "Mean forcing")

ax[0].plot(t1s, std_f1s, marker = "^", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "red", markeredgecolor="red", color="red", label = "Std forcing")

ax[0].set_xlabel(rf'Time (s)', fontsize = 14, labelpad = 5)
ax[0].set_ylabel(f'Forcing (UA)', fontsize = 14, labelpad = 5)
ax[0].set_title(f"Sliding moments over {Tint}s", fontsize=14, pad=20)

ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].tick_params(axis='both', which='minor', labelsize=12)

ax[0].legend(loc = "upper left")

ax[0].grid()

ax[1].plot(t1s, mean_p1s, marker = "o", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "black", markeredgecolor="black", color="black", label = "Mean forcing")

ax[1].plot(t1s, std_p1s, marker = "^", markersize=2, linewidth = 1, linestyle = "-",
        markerfacecolor = "red", markeredgecolor="red", color="red", label = "Std forcing")

ax[1].set_xlabel(rf'Time (s)', fontsize = 14, labelpad = 5)
ax[1].set_ylabel('Pressure (Pa)', fontsize = 14, labelpad = 5)

ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='minor', labelsize=12)

ax[1].legend(loc = "upper left")

ax[1].grid()

##########################################
## Saving figure here
##########################################
fig.savefig('./figures/gong_moy'+str(1000*Tint)+'.png',  bbox_inches = 'tight')

### Question - 2

Follow the tasks below,

1. Calculate the pressure response signal spectrum for the full signal (K = 1).
2. By playing with the frequency zoom and the scale (lin or log) of the DSP, comment on the content of the pressure response signal.

In [ ]:
# Full signal, no average
Nfft = 2**21
# frequency resolution
df = Fs/Nfft
Nwindow = Nfft
Noverlap = int(Nfft/2)

# Estimation
F,Pxx = welch(np.array(Psig[:Nwindow]),
              Fs,
              window = 'hamming',
              nperseg = Nwindow,
              # noverlap = Noverlap,
              nfft = Nfft)

##########################################
## Plotting routine
##########################################
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,5))

ax[0].plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df:.3f}")

ax[0].set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax[0].set_ylabel(r"DSP ($m^2 s^{-2} Hz^{-1}$) ", fontsize = 14, labelpad = 5)
ax[0].set_title('Power Spectral Density: gong - Linear scale', fontsize=14, pad=20)

ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].tick_params(axis='both', which='minor', labelsize=12)

ax[0].grid()

# ax[0].set_xlim()
# ax[0].set_ylim()

ax[1].plot(F, Pxx, marker = ".", markersize=2, linewidth = 1, 
        markerfacecolor = "tab:blue", markeredgecolor="tab:blue", color="tab:blue", label = rf"$\Delta f$={df:.3f}")

ax[1].set_xlabel("Frequency (Hz)", fontsize = 14, labelpad = 5)
ax[1].set_ylabel(r"DSP ($m^2 s^{-2} Hz^{-1}$) ", fontsize = 14, labelpad = 5)
ax[1].set_title('Power Spectral Density: gong - log-log scale', fontsize=14, pad=20)

ax[1].tick_params(axis='both', which='major', labelsize=12)
ax[1].tick_params(axis='both', which='minor', labelsize=12)

ax[1].set_xscale("log")
ax[1].set_yscale("log")

ax[1].grid()

# ax[1].set_xlim()
# ax[1].set_ylim()

fig.savefig(f'./figures/gong_{df}_{Nfft}.png',  bbox_inches = 'tight')

### Question - 3

Calculate the spectrum of the pressure response signal and that of the forcing at 5 Hz resolution without overlap.
1. What is the associated temporal resolution ?
2. Is it possible to increase the time resolution ?
3. Study the pressure response. What is happening in the system ?
4. Study the forcing. Do we find what we would have expected ? If not, suggest an explanation.

In [ ]:
# spectrogram of pressure response
# Frequency resolution
df = 5
# FFT point number
Nfft = int(Fs/df)
# window point number
Nwindow = Nfft
Noverlap = Nfft/2
# number of FFT segments
K = int(Nsamples/Nfft)
# time resolution
dt_specgram = Nwindow * Ts

# estimate spectrograms in each segment and corresponding times
SpecgramP = np.zeros((int(Nfft/2)+1,K)) # pressure
SpecgramF = np.zeros((int(Nfft/2)+1,K)) # Forcing

tspecgram = np.zeros(K)
# loop on segments
for i in range(1,K):
   # Pressure
   freq_specgramP,SpecgramP[:,i] = welch(Psig[(i-1)*Nwindow+1:i*Nwindow],
                                         Fs,
                                         window = 'hamming',
                                         nperseg = Nwindow,
                                         noverlap = Noverlap,
                                         nfft = Nfft)
   # FOrcing
   freq_specgramF,SpecgramF[:,i] = welch(Fsig[(i-1)*Nwindow+1:i*Nwindow],
                                         Fs,
                                         window = 'hamming',
                                         nperseg = Nwindow,
                                         noverlap = Noverlap,
                                         nfft = Nfft)
   # mid time of a segment
   tspecgram[i] = (i-0.5)*Nwindow*Ts 

# Plot for pressure
fig, ax = plt.subplots()
pcm = ax.pcolor(
    tspecgram[1:], 
    freq_specgramP[:-1] / 1000, 
    np.log(SpecgramP[:-1, 1:])  # remove zero term to avoid log(-inf)
)
ax.set_xlabel("t (s)")
ax.set_ylabel("f (kHz)")
ax.set_title("Pressure response spectrogram")
fig.colorbar(pcm, ax=ax)
ax.set_ylim([0, 4])
fig.savefig("./figures/gong_spectrogram_P.png")

# Plot for forcing
fig, ax = plt.subplots()
pcm = ax.pcolor(
    tspecgram[1:], 
    freq_specgramF[:-1] / 1000, 
    np.log(SpecgramF[:-1, 1:])
)
ax.set_xlabel("t (s)")
ax.set_ylabel("f (kHz)")
ax.set_title("Spectrogramme du signal de réponse en forçage")
fig.colorbar(pcm, ax=ax)
ax.set_ylim([0, 4])
fig.savefig("./figures/gong_spectrogramme_F.png")

